In [32]:
import numpy as np
import tensorflow as tf 
import csv
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [20]:
data =  pd.read_csv('data_t1.csv')
data = shuffle(data)
data.shape

(5748, 1202)

In [80]:
train, test =train_test_split(data, test_size=0.2)

train = np.asarray(train)
test = np.asarray(test)

xtr = train[:, 0:1201]
ytr = train[:, 1201]

xts = test[:, 0:1201]
yts = test[:, 1201]

In [67]:
n_classes = 8 

label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(n_classes))

yohtr = label_binarizer.transform(ytr)
yohts = label_binarizer.transform(yts)


In [68]:
yohtr.shape

(4598, 8)

In [101]:
tr = int(0.8*data.shape[0])

n_nodes_hl1 = 50
n_nodes_hl2 = 50
n_nodes_hl3 = 50

n_classes = 8
batch_size = 200 # the number of training examples in one forward/backward pass

x = tf.placeholder('float' ,[None,1201])
y = tf.placeholder('float',[None,n_classes])

In [102]:
def neural_network_model(data):

    # initializing weights and biases for the hidden layers and the output layer
    hidden_l1 = {'weights': tf.Variable(tf.random_normal([1201,n_nodes_hl1])), 
                 'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_l2 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])), 
                 'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_l3 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])), 
                 'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_l = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])), 
                 'biases':tf.Variable(tf.random_normal([n_classes]))}

    # computing the ouput of each layer and applying a non-linearity at the end of each layer computation
    l1 = tf.add(tf.matmul(data, hidden_l1['weights']), hidden_l1['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1, hidden_l2['weights']), hidden_l2['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2, hidden_l3['weights']), hidden_l3['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.add(tf.matmul(l3, output_l['weights']), output_l['biases'])

    return output


In [103]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

    # learning rate = 0.001
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    # one epoch = one cycle of feed-forward and backprop
    n_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(n_epochs):
            epoch_loss = 0
            for index, offset in enumerate(range(0, tr, batch_size)):
                x_epoch, y_epoch = np.array(xtr[offset:offset + batch_size,:]), np.array(yohtr[offset:offset + batch_size])
                _, c = sess.run([optimizer, cost], feed_dict ={x:x_epoch, y:y_epoch}) 
                epoch_loss +=c
            print ('Epoch', epoch, 'completed out of', n_epochs, 'loss:', epoch_loss)

        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy  = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x:xts, y:yohts}))
        
#         prediction_fin=tf.argmax(prediction,1)
#         test_pred = (np.asarray(sess.run([prediction_fin],feed_dict={x:xts})).T)
                              
#         pred_pd = pd.read_csv(csv_test, index_col = 0)
#         pred_pd['label'] =test_pred
#         pred_pd.to_csv(csv_predicted)
#         print('Test:',test_pred)
        
train_neural_network(x)

Epoch 0 completed out of 10 loss: 0.0
Epoch 1 completed out of 10 loss: 0.0
Epoch 2 completed out of 10 loss: 0.0
Epoch 3 completed out of 10 loss: 0.0
Epoch 4 completed out of 10 loss: 0.0
Epoch 5 completed out of 10 loss: 0.0
Epoch 6 completed out of 10 loss: 0.0
Epoch 7 completed out of 10 loss: 0.0
Epoch 8 completed out of 10 loss: 0.0
Epoch 9 completed out of 10 loss: 0.0
Accuracy: 0.0252174
